<a href="https://colab.research.google.com/github/Mechanics-Mechatronics-and-Robotics/CV-2025/blob/main/Lab_1_Feature_Extraction_and_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab. \#1. Feature Extraction and Machine Learning

## Problem Statement

The lab deals with comparison of two approaches to machine learning (ML) and computer vision (CV). The first approach is processing of hand-designed features, e.g. geometric features of objects in images, with an ML classification model. The second approach is using of the ML model for both, the automatic feature extraction and the following classification.

The MNIST database of handwritten digits has a training set of 60,000 examples, and a test set of 10,000 examples.

The hand-designed features can be extracted with standart tools in [scikit-learn](https://scikit-learn.org/1.5/modules/feature_extraction.html)

## Tasks and Requirements
* Check the [Linghtning framework](https://lightning.ai/docs/pytorch/stable/) (Level Up, Core API, Optional API section of the manual)
* Check the [ClearML](https://clear.ml/docs/latest/docs/integrations/pytorch_lightning/)
* Fill the table in the Results section and fill the Conclusion section

Bonus

* apply a t-SNE model to visualize both, the original images dataset, and the designed table dataset with hand-extracted features from the images

# Import and Install Libraries

In [19]:
!pip install pytorch-lightning clearml

In [20]:
#Pytorch modules
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import MNIST
from torchvision import datasets, transforms

#Numpy
import numpy as np

#Pandas
import pandas as pd

#Lightning & logging
import pytorch_lightning as pl
from pytorch_lightning import Trainer

#Data observation
import os
from pathlib import Path

#Plotting
import matplotlib.pyplot as plt
import seaborn as sns

#Logging
from clearml import Task

# Set the Model

## Simulation Settings

Check the current directory

In [21]:
os.getcwd() #returns the current working directory

'/content'

Paths and initializations of the weights

In [22]:
# # Path to the folder where the dataset is saved
# DATASET_PATH = os.environ.get("PATH_DATASET", "data/")
# print(f'DATASET_PATH: {DATASET_PATH}')

# Path to the folder where the trained or pretrained models are saved
CHECKPOINT_PATH = os.environ.get("PATH_CHECKPOINT", "saved_models")
print(f'CHECKPOINT_PATH: {CHECKPOINT_PATH}')

os.makedirs(DATASET_PATH, exist_ok=True) #create the directory
os.makedirs(CHECKPOINT_PATH, exist_ok=True) #create the directory

isINFERENCE = False # inference mode with downloading the saved weights
isPretrained = False # use the pretrained weights when training

# Function for setting the seed to implement parallel tests
seed = 42 # random seeds are 42, 0, 17, 9, 3
pl.seed_everything(seed)

# Ensure that all operations are deterministic on GPU (if used) for reproducibility
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

INFO:lightning_fabric.utilities.seed:Seed set to 42


DATASET_PATH: data/
CHECKPOINT_PATH: saved_models


## Logging

To configure ClearML in your Colab environment, follow these steps:

---

### *Step 1: Create a ClearML Account*
1. Go to the [ClearML website](https://clear.ml/).
2. Sign up for a free account if you don’t already have one.
3. Once registered, log in to your ClearML account.

---

### *Step 2: Get Your ClearML Credentials*
1. After logging in, navigate to the **Settings** page (click on your profile icon in the top-right corner and select **Settings**).
2. Under the **Workspace** section, find your **+ Create new credentials**.
3. Copy these credentials for a Jupiter notebook into the code cell below.

---

### *Step 3: Accessing the ClearML Dashboard*
1. Go to your ClearML dashboard (https://app.clear.ml).
2. Navigate to the **Projects** section to see your experiments.
3. Click on the experiment (e.g., `Lab_1`) to view detailed metrics, logs, and artifacts.

---

In [23]:
#Enter your code here to implement Step 2 as it is shown below
%env CLEARML_WEB_HOST=https://app.clear.ml/
%env CLEARML_API_HOST=https://api.clear.ml
%env CLEARML_FILES_HOST=https://files.clear.ml
%env CLEARML_API_ACCESS_KEY=ZP02U03C6V5ER4K9VWRNZT7EWA5ZTV
%env CLEARML_API_SECRET_KEY=BtA5GXZufr6QGpaqhX1GSKPTvaCt56OLqaNqUGLNoxx2Ye8Ctwbui0Ln5OXVnzUgH4I

In [ ]:
task = Task.init(project_name="CV-2025", task_name="Lab_1")
print("ClearML is configured correctly!")

## Dataset

Summary

In [24]:
DATASET = 'MNIST'
ns = {'train': 55000, 'val': 5000, 'test': 10000}

SIZE = 28 #image size
NUM_CLASSES = 10
CLASS_NAMES = ['zero' ,'one', 'two', 'three', 'four',
               'five', 'six', 'seven', 'eight', 'nine']

Normalization parameters

In [25]:
mean = np.array([0.1307])
std  = np.array([0.3081])

Transforms

In [26]:
# data_transforms = transforms.Compose([transforms.ToTensor(),
#                               transforms.Normalize(mean, std)])

## Collect hyperparameters

In [27]:
#Model parameters
LOSS_FUN = 'CE'
ARCHITECTURE = 'MLP'
lr = 0.0001 #
n = 100 # number of epochs

batch_size = 64
num_workers = 8

optimization_algorithm = 'Adam'# 'SGD','Adam'
MOMENTUM = 0.9
WD = 1e-4 # weight decay (L_2 regularization), default value for 'Adam' is 0

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
roundRun = 5 #number of digits in the results performance

#Visualization
figSize = (8,8)
nSamples = 4
numBins = 50

#Summary: hyperparameters
hyperparameters = {
    "seed": seed,
    "lr": lr,
    "isINFERENCE": isINFERENCE,
    "isPretrained": isPretrained,
    "dataset": DATASET,
    "n_classes": NUM_CLASSES,
    "class_names": CLASS_NAMES,
    "bs": batch_size,
    "num_workers": num_workers,
    "num_epochs": n,
    "model_filename": ARCHITECTURE,
    "optimization_algorithm": optimization_algorithm,
    "momentum": MOMENTUM,
    "criterion": LOSS_FUN,
    "weight decay": WD,
    "device": DEVICE,
    "fig_size": figSize,
    "num_samples": nSamples,
    "num_bins": numBins,
}

50000

# Functions

## Lightning

Data module

In [42]:
class MNISTDataModule(pl.LightningDataModule):
    def __init__(self, batch_size=hyperparameters['bs'], mean=mean, std=std,
                 ns=ns):
        super().__init__()
        self.batch_size = batch_size
        self.mean = mean
        self.std = std
        self.ns = ns
        self.transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(self.mean, self.std)
        ])

    def prepare_data(self):
        # Download MNIST dataset
        datasets.MNIST(root='./data', train=True, download=True)
        datasets.MNIST(root='./data', train=False, download=True)

    def setup(self, stage=None):
        # Split dataset into train and validation sets
        mnist_full = datasets.MNIST(root='./data', train=True, transform=self.transform)
        self.mnist_train, self.mnist_val = random_split(mnist_full, [self.ns['train'], self.ns['val']])
        self.mnist_test = datasets.MNIST(root='./data', train=False, transform=self.transform)

    def train_dataloader(self):
        return DataLoader(self.mnist_train, batch_size=self.batch_size, shuffle=True)

    def val_dataloader(self):
        return DataLoader(self.mnist_val, batch_size=self.batch_size)

    def test_dataloader(self):
        return DataLoader(self.mnist_test, batch_size=self.batch_size)

Training module

Callbacks

## Model

MLP

In [28]:
class MLP(pl.LightningModule):

  def __init__(self):
    super(MLP, self).__init__()

    # mnist images are (1, 28, 28) (channels, width, height)
    self.layer_1 = torch.nn.Linear(SIZE * SIZE, 128)
    self.layer_2 = torch.nn.Linear(128, 256)
    self.layer_3 = torch.nn.Linear(256, NUM_CLASSES)

  def forward(self, x):
    batch_size, channels, width, height = x.siz()

    # (b, 1, SIZE, SIZE) -> (b, 1*SIZE*SIZE)
    x = x.view(batch_size, -1)

    # layer 1
    x = self.layer_1(x)
    x = torch.relu(x)

    # layer 2
    x = self.layer_2(x)
    x = torch.relu(x)

    # layer 3
    x = self.layer_3(x)

    return x

## Loss

Create a loss function class, or use a standart one.

In [29]:
# Cross entropy loss
class CEloss(nn.Module):
    def __init__(self):
        super(CEloss, self).__init__()

    def forward(self,x,y):
        prob = nn.functional.softmax(x,1)
        log_prob = -1.0 * torch.log(prob)
        loss = log_prob.gather(1, y.unsqueeze(1))
        loss = loss.mean()
        return loss

## Visualization

In [30]:
def imshow(inp, title):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    inp = ((std * inp) + mean)
    inp = np.clip(inp, 0, 1)
    #plt.grid(visible=None, which='major',axis='both')
    plt.axis('off')
    plt.imshow(inp)
    plt.title(title)
    plt.show()

In [31]:
def hist(df, values, histSize):
    sns.set(style="darkgrid")
    sns.set(rc={'figure.figsize': histSize})

    n = hyperparameters['num_bins']

    sns.histplot(data=df[df['True or false prediction'] == True], x = values, color="skyblue",
                 label='True predictions',  bins=n, kde=True)
    sns.histplot(data=df[df['True or false prediction'] == False],x = values, color="red",
                 label='False predictions', bins=n, kde=True)
    plt.legend()
    return

In [35]:
# %% [markdown]
# # MNIST Classification with PyTorch Lightning and ClearML
#
# This notebook demonstrates how to classify MNIST digits using PyTorch Lightning and log experiments with ClearML.

# %% [markdown]
# ## Step 1: Set Up ClearML Task
#
# ClearML helps in tracking experiments, logging metrics, and storing models.

# %% [markdown]
# ## Step 2: Define the Data Module
#
# We'll use PyTorch Lightning's `LightningDataModule` to handle data loading and preprocessing.

# %%
class MNISTDataModule(pl.LightningDataModule):
    def __init__(self, batch_size=64):
        super().__init__()
        self.batch_size = batch_size
        self.transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.1307,), (0.3081,))
        ])

    def prepare_data(self):
        # Download MNIST dataset
        datasets.MNIST(root='./data', train=True, download=True)
        datasets.MNIST(root='./data', train=False, download=True)

    def setup(self, stage=None):
        # Split dataset into train and validation sets
        mnist_full = datasets.MNIST(root='./data', train=True, transform=self.transform)
        self.mnist_train, self.mnist_val = random_split(mnist_full, [55000, 5000])
        self.mnist_test = datasets.MNIST(root='./data', train=False, transform=self.transform)

    def train_dataloader(self):
        return DataLoader(self.mnist_train, batch_size=self.batch_size, shuffle=True)

    def val_dataloader(self):
        return DataLoader(self.mnist_val, batch_size=self.batch_size)

    def test_dataloader(self):
        return DataLoader(self.mnist_test, batch_size=self.batch_size)

# %% [markdown]
# ## Step 3: Define the Model
#
# We'll create a simple convolutional neural network (CNN) for MNIST classification.

# %%
class MNISTModel(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)
        self.dropout = nn.Dropout(0.5)
        self.loss_fn = nn.CrossEntropyLoss()

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 64 * 7 * 7)
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.loss_fn(y_hat, y)
        self.log('train_loss', loss)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.loss_fn(y_hat, y)
        self.log('val_loss', loss)
        return loss

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.loss_fn(y_hat, y)
        self.log('test_loss', loss)
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.001)

# %% [markdown]
# ## Step 4: Train the Model
#
# We'll use PyTorch Lightning's `Trainer` to handle training and validation.

# %%
# Initialize data module and model
data_module = MNISTDataModule()
model = MNISTModel()

# Initialize PyTorch Lightning Trainer
trainer = Trainer(max_epochs=2, accelerator="auto", devices="auto")

# Train the model
trainer.fit(model, data_module)

# %% [markdown]
# ## Step 5: Test the Model
#
# After training, we'll evaluate the model on the test set.

# %%
# Test the model
trainer.test(model, datamodule=data_module)

# %% [markdown]
# ## Step 6: Log Model with ClearML
#
# Finally, we'll log the trained model using ClearML.

# %%
# Log the trained model
task.update_output_model(model_path="mnist_model.pth")
torch.save(model.state_dict(), "mnist_model.pth")

# %% [markdown]
# ## Conclusion
#
# This notebook demonstrated how to:
# 1. Set up a ClearML task for experiment tracking.
# 2. Use PyTorch Lightning for data loading and model training.
# 3. Train and test a simple CNN on the MNIST dataset.
# 4. Log the trained model using ClearML.
#
# You can now explore the ClearML dashboard to analyze metrics, compare experiments, and manage models.

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.callbacks.model_summary:
  | Name    | Type             | Params | Mode 
-----------------------------------------------------
0 | conv1   | Conv2d           | 320    | train
1 | conv2   | Conv2d           | 18.5 K | train
2 | pool    | MaxPool2d        | 0      | train
3 | fc1     | Linear           | 401 K  | train
4 | fc2     | Linear           | 1.3 K  | train
5 | dropout | Dropout          | 0      | train
6 | loss_fn | CrossEntropyLoss | 0      | train
-----------------------------------------------------
421 K     Trainable params
0         Non-trainable params
421 K     Total params
1.687     Total estimated model params size (MB)
7         Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=2` reached.


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │    0.03598685562610626    │
└───────────────────────────┴───────────────────────────┘

2025-01-06 19:50:29,188 - clearml.storage - ERROR - Exception encountered while uploading [Errno 2] No such file or directory: 'mnist_model.pth'
2025-01-06 19:50:29,194 - clearml.Task - INFO - Failed model upload


In [43]:
task.close()